In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

import torch
from transformers import BertTokenizer, BertModel

import logging
logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv("/content/drive/MyDrive/malware_data.csv", index_col = 0)

In [4]:
data.head()

,OpCode Sequence,Malware Family
0,push mov push mov mov mov mov cmp jz push sti ...,CLUSTERclarkclark
1,push add mov push call test jz movzx mov add p...,ufasoftbitcoin
2,push push push call add push call mov push pus...,ufasoftbitcoin
3,push add mov push call test jz movzx mov add p...,ufasoftbitcoin
4,mov push mov push push mov and test push push ...,CLUSTERgdata


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True)
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [18]:
all_embeddings = dict()
token_occurences = dict()
for index, row in data.iterrows():
  opcodes = row['OpCode Sequence']
  marked_opcodes = "[CLS] " + opcodes + " [SEP]"
  tokenized_opcodes = tokenizer.tokenize(marked_opcodes)
  tokenized_opcodes_arr = []
  if len(tokenized_opcodes) > 512:
    iter = 0
    while(iter*512 < len(tokenized_opcodes)):
      if tokenized_opcodes[iter*512] != '[CLS]':
        tokenized_opcodes.insert(iter*512, '[CLS]')
      if tokenized_opcodes[min((iter+1)*512 - 1, len(tokenized_opcodes) - 1)] != '[SEP]':
        tokenized_opcodes.insert(min((iter+1)*512 - 1, len(tokenized_opcodes) - 1), '[SEP]')
      iter = iter + 1
    while iter*512 < len(tokenized_opcodes):
      tokenized_opcodes_arr.append(tokenized_opcodes[iter*512:min((iter+1)*512, len(tokenized_opcodes))])
  else:
    tokenized_opcodes_arr.append(tokenized_opcodes)
  for batch in tokenized_opcodes_arr:
    print(batch)
    indexed_tokens = tokenizer.convert_tokens_to_ids(batch)
    segments_ids = [1] * len(batch)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
      outputs = model(tokens_tensor, segments_tensors)
      hidden_states = outputs[2]
      token_embeddings = torch.stack(hidden_states, dim=0)
      token_embeddings = torch.squeeze(token_embeddings, dim=1)
      token_embeddings = token_embeddings.permute(1,0,2)
      token_vecs_sum = []
      for idx, token in enumerate(token_embeddings):
        sum_vec = torch.sum(token[-4:], dim=0)
        if batch[idx] in all_embeddings:
          all_embeddings[batch[idx]] = all_embeddings[batch[idx]].add(sum_vec)
          token_occurences[batch[idx]] = token_occurences[batch[idx]] + 1
        else:
          all_embeddings[batch[idx]] = sum_vec
          token_occurences[batch[idx]] = 1

['[CLS]', 'push', 'mo', '##v', 'push', 'mo', '##v', 'mo', '##v', 'mo', '##v', 'mo', '##v', 'cm', '##p', 'j', '##z', 'push', 'st', '##i', 'outs', '##d', 'fi', '##ld', 'push', 'push', 'sb', '##b', 'push', 'call', 'rc', '##l', 'outs', '##d', 're', '##t', '##n', 'fist', '##tp', 'sub', 'pop', 're', '##t', '##n', 'push', '##a', 'mo', '##v', 'lea', 'push', 'or', 'j', '##mp', 'mo', '##v', 'inc', 'mo', '##v', 'inc', 'add', 'j', '##nz', 'mo', '##v', 'sub', 'ad', '##c', 'j', '##b', 'mo', '##v', 'add', 'j', '##nz', 'mo', '##v', 'sub', 'ad', '##c', 'ad', '##c', 'add', 'j', '##nb', 'j', '##nz', 'mo', '##v', 'sub', 'ad', '##c', 'j', '##nb', 'x', '##or', 'sub', 'j', '##b', 'sh', '##l', 'mo', '##v', 'inc', 'x', '##or', 'j', '##z', 'mo', '##v', 'add', 'j', '##nz', 'mo', '##v', 'sub', 'ad', '##c', 'ad', '##c', 'add', 'j', '##nz', 'mo', '##v', 'sub', 'ad', '##c', 'ad', '##c', 'j', '##nz', 'inc', 'add', 'j', '##nz', 'mo', '##v', 'sub', 'ad', '##c', 'ad', '##c', 'add', 'j', '##nb', 'j', '##nz', 'mo', '##v',

In [20]:
for key in all_embeddings:
  all_embeddings[key] = torch.div(all_embeddings[key], token_occurences[key])

In [23]:
print(all_embeddings['mo'].size())

torch.Size([768])


In [25]:
import pickle

with open('bert_embeddings.pkl', 'wb') as fp:
  pickle.dump(all_embeddings, fp)